# 大众点评评价情感分析~
先上结果：

| 糖水店的评论文本                             | 模型预测的情感评分 |
| :------------------------------------------- | :----------------- |
| '糖水味道不错，滑而不腻，赞一个，下次还会来' | 0.91               |
| '味道一般，没啥特点'                         | 0.52               |
| '排队老半天，环境很差，味道一般般'           | 0.05               |

模型的效果还可以的样子，yeah~接下来我们好好讲讲怎么做的哈，我们通过爬虫爬取了大众点评广州8家最热门糖水店的5W条评论信息以及评分作为训练数据，前面的分析我们得知样本很不均衡。接下来我们的整体思路就是：文本特征处理(分词、去停用词、TF-IDF)—机器学习建模—模型评价。

我们先不处理样本不均衡问题，直接建模后查看结果，接下来我们再按照两种方法处理样本不均衡，对比结果。

### 数据读入和浏览

In [2]:
import pandas as pd
from matplotlib import pyplot as plt
import jieba
data = pd.read_csv('data.csv')
data.head()

,cus_id,comment_time,comment_star,cus_comment,kouwei,huanjing,fuwu,shopID,stars,year,month,weekday,hour,comment_len
0,迷糊泰迪,2018-09-20 06:48:00,sml-str40,南信 算是 广州 著名 甜品店吧 ，好几个 时间段 路过 ，都是 座无虚席。 看着 餐单 ...,非常好,好,好,518986.0,4.0,2018.0,9.0,3.0,6.0,184.0
1,稱霸幼稚園,2018-09-22 21:49:00,sml-str40,中午吃完了所谓的早茶 回去放下行李 休息了会 就来吃下午茶了[服务]两层楼 楼下只能收现金...,很好,很好,很好,518986.0,4.0,2018.0,9.0,5.0,21.0,266.0
2,爱吃的美美侠,2018-09-22 22:16:00,sml-str40,【VIP冲刺王者战队】【吃遍蓉城战队】【VIP有特权】五月份和好朋友毕业旅行来了广州。我们都...,很好,很好,很好,518986.0,4.0,2018.0,9.0,5.0,22.0,341.0
3,姜姜会吃胖,2018-09-19 06:36:00,sml-str40,都说来广州吃糖水就要来南信招牌姜撞奶，红豆双皮奶牛三星，云吞面一楼现金，二楼微信支付宝位置不...,非常好,很好,很好,518986.0,4.0,2018.0,9.0,2.0,6.0,197.0
4,forevercage,2018-08-24 17:58:00,sml-str50,一直很期待也最爱吃甜品，广州的甜品很丰富很多样，来之前就一直想着一定要过来吃到腻，今天总算实...,非常好,很好,很好,518986.0,5.0,2018.0,8.0,4.0,17.0,261.0


### 构建标签值

大众点评的评分分为1-5分，1-2为差评，4-5为好评，3为中评，因此我们把1-2记为0,4-5记为1,3为中评，对我们的情感分析作用不大，丢弃掉这部分数据，但是可以作为训练语料模型的语料。我们的情感评分可以转化为标签值为1的概率值，这样我们就把情感分析问题转为文本分类问题了。

In [5]:
#构建label值
def zhuanhuan(score):
    if score > 3:
        return 1
    elif score < 3:
        return 0
    else:
        return None
    
#特征值转换
data['target'] = data['stars'].map(lambda x:zhuanhuan(x))
data_model = data.dropna()

### 文本特征处理

In [8]:
#切分测试集、训练集
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data_model['cus_comment'], data_model['target'], random_state=1, test_size=0.25)

In [9]:
#引入停用词
infile = open("stopwords.txt",encoding='utf-8')
stopwords_lst = infile.readlines()
stopwords = [x.strip() for x in stopwords_lst]

In [10]:
#中文分词
def fenci(train_data):
    words_df = train_data.apply(lambda x:' '.join(jieba.cut(x)))
    return words_df
 
x_train_fenci = fenci(x_train)
x_train_fenci[:5]

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\bin\AppData\Local\Temp\jieba.cache
Loading model cost 0.815 seconds.
Prefix dict has been built succesfully.


16464    简单 的 店面 ， 甜品 种类 超多 ， 选择 困难 ， 点 了 红豆 薏米 芋头 ， 还 ...
28132    打算 去 长隆 玩水 在 番禺 附近 搜到 的 店 ， 离住 的 地方 很近 ， 大热天 走...
14213    甜品 第一站 当然 是 选 在 百花 甜品 了 ， 光听 这个 名字 就 可以 想象 种类 ...
24829    在 大众 点评 看到 推荐 的 同福路 美食 ， 刚刚 和 朋友 闲逛 来到 同福路 ， 决...
31055    几乎 每次 去 广州 都 会 去 这间 店 吃 甜品 ， 非常 不错 的 一间 店 ， 环境...
Name: cus_comment, dtype: object

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(stop_words=stopwords, max_features=3000, ngram_range=(1,2))
tv.fit(x_train_fenci)

D:\SOFTWARE\Anaconda\lib\site-packages\sklearn\feature_extraction\text.py:286: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'aren', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'lex', 'll', 'mon', 'null', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn', '１２', 'ｌｉ', 'ｚｘｆｉｔｌ'] not in stop_words.
  sorted(inconsistent))


TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=3000, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '--', '.', '..', '...', '......', '...................', './', '.一', '记者', '数', '年', '月', '日', '时', '分', '秒', '/', '//', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '://', '::', ';', '<', '=', '>', '>>', '?', '@'...3', '94', '95', '96', '97', '98', '99', '100', '01', '02', '03', '04', '05', '06', '07', '08', '09'],
        strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

### 机器学习建模

这里我们使用文本分类的经典算法朴素贝叶斯算法

In [12]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(tv.transform(fenci(x_train)), y_train)
classifier.score(tv.transform(fenci(x_test)), y_test)

0.9319690265486725

In [13]:
from sklearn.metrics import roc_auc_score, f1_score
y_pred = classifier.predict_proba(tv.transform(fenci(x_test)))[:,1]
roc_auc_score(y_test,y_pred)

0.8968929292125364

可以看出，准确率和AUC值都非常不错的样子，但效果是否真的好呢？我们查看一下**混淆矩阵**

In [16]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_predict)
cm

array([[  68,  365],
       [   4, 4987]], dtype=int64)

可以看出，负类的预测非常不准，433单准确预测为负类的只有15.7%，应该是由于数据不平衡导致的，接下来我们就解决一下这个问题

### 过采样（单纯复制）

简单粗暴的方法是复制少数样本，缺点是虽然引入了额外的训练数据，但没有给少数类样本增加任何新的信息，非常容易造成过拟合。

In [18]:
data['target'].value_counts()

1.0    19915
0.0     1779
Name: target, dtype: int64

In [19]:
#把0类样本复制10次
index_tmp = y_train==0
y_tmp = y_train[index_tmp]
x_tmp = x_train[index_tmp]
x_train2 = pd.concat([x_train,x_tmp,x_tmp,x_tmp,x_tmp,x_tmp,x_tmp,x_tmp,x_tmp,x_tmp,x_tmp])
y_train2 = pd.concat([y_train,y_tmp,y_tmp,y_tmp,y_tmp,y_tmp,y_tmp,y_tmp,y_tmp,y_tmp,y_tmp])

In [20]:
clf2 = MultinomialNB()
clf2.fit(tv.transform(fenci(x_train2)), y_train2)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [21]:
y_pred2 = clf2.predict_proba(tv.transform(fenci(x_test)))[:,1]
roc_auc_score(y_test,y_pred2)

0.9064126050447385

In [18]:
y_predict2 = clf2.predict(tv.transform(fenci(x_test)))
cm = confusion_matrix(y_test, y_predict2)
cm

array([[ 333,  100],
       [ 625, 4366]])

可以看出，即使是简单粗暴的复制样本来处理样本不平衡问题，负样本的识别率大幅上升了，变为77%，慢慢的幸福感呀~我们自己写两句评语来看看

In [24]:
def fenxi(strings):
    strings_fenci = fenci(pd.Series([strings]))
    return float(clf2.predict_proba(tv.transform(fenci(strings_fenci)))[:,1])

In [26]:
fenxi('排队人太多，环境不好，口味一般')

0.32001183981270204

可以看出把0类别的识别出来了，太棒了~

### 过采样（SMOTE算法）

SMOTE（Synthetic minoritye over-sampling technique,SMOTE），是通过对少数样本进行插值来获取新样本的。比如对于每个少数类样本a，从 a最邻近的样本中选取 样本b，然后在对 ab 中随机选择一点作为新样本。

对SMOTE感兴趣的同学可以看下这篇文章https://www.jianshu.com/p/ecbc924860af

In [40]:
from imblearn.over_sampling import SMOTE
oversampler=SMOTE(random_state=0)
x_train_vec = tv.transform(fenci(x_train))
x_resampled, y_resampled = oversampler.fit_sample(x_train_vec, y_train)

In [42]:
y_train.value_counts()

1.0    14923
0.0     1346
Name: target, dtype: int64

In [41]:
pd.Series(y_resampled).value_counts()

0.0    14923
1.0    14923
dtype: int64

使用全部数据进行训练

In [21]:
tv2 = TfidfVectorizer(stop_words=stopwords, max_features=3000, ngram_range=(1,2))
tv2.fit(data_model['cus_comment'])
clf = MultinomialNB()
clf.fit(tv2.transform(fenci(data_model['cus_comment'])), data_model['target'])

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)